# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 9 2022 1:10PM,252740,10,0086237403,ISDIN Corporation,Released
1,Dec 9 2022 1:10PM,252740,10,0086237404,ISDIN Corporation,Released
2,Dec 9 2022 1:10PM,252740,10,0086237405,ISDIN Corporation,Released
3,Dec 9 2022 1:10PM,252740,10,0086237481,ISDIN Corporation,Released
4,Dec 9 2022 1:09PM,252739,16,SSF_AirCargo_12/8/22,Sartorius Stedim Filters Inc.,Released
5,Dec 9 2022 12:55PM,252738,19,ELI12132022,Eli Lilly and Company,Released
6,Dec 9 2022 12:37PM,252735,15,30012775,"Alliance Pharma, Inc.",Released
7,Dec 9 2022 12:37PM,252735,15,30012776,"Alliance Pharma, Inc.",Released
8,Dec 9 2022 12:37PM,252735,15,30012777,"Alliance Pharma, Inc.",Released
9,Dec 9 2022 12:37PM,252735,15,30012778,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
48,252734,Released,1
49,252735,Released,4
50,252738,Released,1
51,252739,Released,1
52,252740,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
252734,NaN,1.0,1.0
252735,NaN,NaN,4.0
252738,NaN,NaN,1.0
252739,NaN,NaN,1.0
252740,NaN,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
252642,0.0,0.0,7.0
252644,0.0,0.0,1.0
252646,0.0,0.0,1.0
252648,0.0,0.0,3.0
252650,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
252642,0,0,7
252644,0,0,1
252646,0,0,1
252648,0,0,3
252650,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,252642,0,0,7
1,252644,0,0,1
2,252646,0,0,1
3,252648,0,0,3
4,252650,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,252642,,,7
1,252644,,,1
2,252646,,,1
3,252648,,,3
4,252650,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 9 2022 1:10PM,252740,10,ISDIN Corporation
4,Dec 9 2022 1:09PM,252739,16,Sartorius Stedim Filters Inc.
5,Dec 9 2022 12:55PM,252738,19,Eli Lilly and Company
6,Dec 9 2022 12:37PM,252735,15,"Alliance Pharma, Inc."
10,Dec 9 2022 12:29PM,252734,10,"Senseonics, Incorporated"
12,Dec 9 2022 12:04PM,252728,19,"AdvaGen Pharma, Ltd"
26,Dec 9 2022 11:57AM,252726,10,ISDIN Corporation
34,Dec 9 2022 11:54AM,252725,10,Bio-PRF
35,Dec 9 2022 11:54AM,252724,10,Bio-PRF
36,Dec 9 2022 11:52AM,252723,19,"GUSA Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Dec 9 2022 1:10PM,252740,10,ISDIN Corporation,,,4
1,Dec 9 2022 1:09PM,252739,16,Sartorius Stedim Filters Inc.,,,1
2,Dec 9 2022 12:55PM,252738,19,Eli Lilly and Company,,,1
3,Dec 9 2022 12:37PM,252735,15,"Alliance Pharma, Inc.",,,4
4,Dec 9 2022 12:29PM,252734,10,"Senseonics, Incorporated",,1,1
5,Dec 9 2022 12:04PM,252728,19,"AdvaGen Pharma, Ltd",,14,
6,Dec 9 2022 11:57AM,252726,10,ISDIN Corporation,,,8
7,Dec 9 2022 11:54AM,252725,10,Bio-PRF,,,1
8,Dec 9 2022 11:54AM,252724,10,Bio-PRF,,1,
9,Dec 9 2022 11:52AM,252723,19,"GUSA Granules USA, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 9 2022 1:10PM,252740,10,ISDIN Corporation,4,,
1,Dec 9 2022 1:09PM,252739,16,Sartorius Stedim Filters Inc.,1,,
2,Dec 9 2022 12:55PM,252738,19,Eli Lilly and Company,1,,
3,Dec 9 2022 12:37PM,252735,15,"Alliance Pharma, Inc.",4,,
4,Dec 9 2022 12:29PM,252734,10,"Senseonics, Incorporated",1,1,
5,Dec 9 2022 12:04PM,252728,19,"AdvaGen Pharma, Ltd",,14,
6,Dec 9 2022 11:57AM,252726,10,ISDIN Corporation,8,,
7,Dec 9 2022 11:54AM,252725,10,Bio-PRF,1,,
8,Dec 9 2022 11:54AM,252724,10,Bio-PRF,,1,
9,Dec 9 2022 11:52AM,252723,19,"GUSA Granules USA, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 9 2022 1:10PM,252740,10,ISDIN Corporation,4,,
1,Dec 9 2022 1:09PM,252739,16,Sartorius Stedim Filters Inc.,1,,
2,Dec 9 2022 12:55PM,252738,19,Eli Lilly and Company,1,,
3,Dec 9 2022 12:37PM,252735,15,"Alliance Pharma, Inc.",4,,
4,Dec 9 2022 12:29PM,252734,10,"Senseonics, Incorporated",1,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 9 2022 1:10PM,252740,10,ISDIN Corporation,4.0,NaN,NaN
1,Dec 9 2022 1:09PM,252739,16,Sartorius Stedim Filters Inc.,1.0,NaN,NaN
2,Dec 9 2022 12:55PM,252738,19,Eli Lilly and Company,1.0,NaN,NaN
3,Dec 9 2022 12:37PM,252735,15,"Alliance Pharma, Inc.",4.0,NaN,NaN
4,Dec 9 2022 12:29PM,252734,10,"Senseonics, Incorporated",1.0,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 9 2022 1:10PM,252740,10,ISDIN Corporation,4.0,0.0,0.0
1,Dec 9 2022 1:09PM,252739,16,Sartorius Stedim Filters Inc.,1.0,0.0,0.0
2,Dec 9 2022 12:55PM,252738,19,Eli Lilly and Company,1.0,0.0,0.0
3,Dec 9 2022 12:37PM,252735,15,"Alliance Pharma, Inc.",4.0,0.0,0.0
4,Dec 9 2022 12:29PM,252734,10,"Senseonics, Incorporated",1.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5053958,87.0,7.0,1.0
15,1010826,18.0,0.0,0.0
16,252739,1.0,0.0,0.0
19,1768908,7.0,14.0,0.0
20,252653,0.0,1.0,0.0
21,4295366,17.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5053958,87.0,7.0,1.0
1,15,1010826,18.0,0.0,0.0
2,16,252739,1.0,0.0,0.0
3,19,1768908,7.0,14.0,0.0
4,20,252653,0.0,1.0,0.0
5,21,4295366,17.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,87.0,7.0,1.0
1,15,18.0,0.0,0.0
2,16,1.0,0.0,0.0
3,19,7.0,14.0,0.0
4,20,0.0,1.0,0.0
5,21,17.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,87.0
1,15,Released,18.0
2,16,Released,1.0
3,19,Released,7.0
4,20,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,1.0,0.0,0.0,0.0,0.0,0.0
Executing,7.0,0.0,0.0,14.0,1.0,0.0
Released,87.0,18.0,1.0,7.0,0.0,17.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0
1,Executing,7.0,0.0,0.0,14.0,1.0,0.0
2,Released,87.0,18.0,1.0,7.0,0.0,17.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0
1,Executing,7.0,0.0,0.0,14.0,1.0,0.0
2,Released,87.0,18.0,1.0,7.0,0.0,17.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()